# Вебинар 1. Введение, примеры задач, бизнес- и ML-метрики 

# Домашнее задание 

**1) Приведите еще примеры метрик для оценки рекомендаций/ранжирования (можно взять из интернета, или ваши знания)**

**2) Доделать все функции, где стоит комментарий "сделать дома"**

## Решение задания 1
***Среднеобратный ранг (MRR@k, Mean Reciprocal Rank)*** - одна из метрик качества ранжирования

- Считаем для первых k рекомендаций
- Найти ранг первого релевантного предсказания $k_u$
- Посчитать reciprocal rank = $\frac{1}{k_u}$ для u-го объекта. Величина, равная обратному рангу первого правильно угаданного элемента.

Среднеобратный ранг определяется как среднее обратных рангов по всем запросам U:

$$MRR = mean(\frac{1}{k_u})$$

*Mean reciprocal rank* изменяется в диапазоне [0,1] и учитывает позицию элементов. Он делает это только для одного элемента — 1-го верно предсказанного, не обращая внимания на все последующие.

Результатом интеракции (взаимодействия пользователя и товара) может быть покупка товара, оценка товара пользователем.

***PFound***

- метрика качества ранжирования, разработанная Яндекс. 

Расчет PFound производится по такой формуле:

$$PFound@K = \sum_{k=1}^KPLook(k)PRel(k),$$


где

*PLook(k)* - вероятность просмотреть k-й документ из списка,

*PRel(k)* - вероятность того, что k-й документ окажется релевантным.

Для оценки вероятности просмотра k-го результата используется два предположения: пользователь просматривает результаты поиска последовательно сверху вниз; он прекращает процесс в случае нахождения релевантного результата либо может остановиться без каких-либо определенных причин ("надоело"). Формула Plook(k) следующая:

$$PLook(k) = PLook(k-1)(1 - PRel(k-1))(1-PBreak), $$

где

*PLook(k-1)* - вероятность того, что пользователь просмотрит (k-1)-ю позицию;

*1 - PRel(k-1)* - вероятность того, что пользователь не
удовлетворится (k-1)-й позицией;

*1-PBreak* - вероятность того, что пользователь не прекратит просмотр по независящим от нас причинам (в модели Яндекс PBreak выбрана равной 0.15).

Выше описан расчет значения PFound в контексте одного
запроса. Целевой задачей процесса машинного обучения является
максимизация усредненного значения метрики по всем запросам
обучающей выборки.  

## Решение задания 2

In [1]:
import pandas as pd
import numpy as np

## 2. ML-mетрики качества

*Хороши ли рекомендации, предлагаемые моделью?*  
Обычно считаются для каждого юзера, затем усредняются по юзерам

Представим, что в магазине всего 10 товаров

In [ ]:
recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [521, 32, 143, 991]

### 1. Hit rate

Hit rate = был ли хотя бы 1 релевантный товар среди рекомендованных

- Иногда применяется, когда продаются достаточно дорогие товары (например, бытовая техника) 

----
Hit rate = (был ли хотя бы 1 релевантный товар среди рекомендованных)   

Hit rate@k = (был ли хотя бы 1 релевантный товар среди топ-k рекомендованных)

In [ ]:
flags = np.isin(bought_list, recommended_list)
flags

array([False, False,  True,  True])

In [ ]:
def hit_rate(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    hit_rate = (flags.sum() > 0) * 1
    
    return hit_rate


def hit_rate_at_k(recommended_list, bought_list, k=5):
    
#   сделать в домашней работе

    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]

    flags = np.isin(bought_list, recommended_list)
    
    hit_rate = (flags.sum() > 0) * 1
    
    return hit_rate

In [ ]:
# проверяем код, сделанный в домашней работе
hit_rate_at_k(recommended_list, bought_list, k=3)

1

In [ ]:
hit_rate(recommended_list, bought_list)

1

### 2. Precision

*Precision* - доля релевантных товаров среди рекомендованных = Какой % рекомендованных товаров  юзер купил

- Пожалуй, самая приближенная к бизнес-метрикам и самая популярная метрика

---

Precision= (# of recommended items that are relevant) / (# of recommended items)  

Precision@k = (# of recommended items @k that are relevant) / (# of recommended items @k)

Money Precision@k = (revenue of recommended items @k that are relevant) / (revenue of recommended items @k)  

**Note:** Обычно k в precision@k достаточно невелико (5-20) и определяется из бизнес-логики. Например, 5 товаров в e-mail рассылке, 20 ответов на первой странице google и т.д

Красная рыба - 400 руб  
Молоко - 60 руб  
Хлеб = 40 руб  
Гречка = 40 руб  
Шоколад = 90 руб  

------  
Варенье - 240 руб  
...  

**Case 1**  
prices_resommended = [400, 60, 40, 40 , 90]  
flags = [1, 0, 0, 0 , 1]  

$precison@5 = \frac{1 + 0 + 0 +0 + 1}{1+1+1+1+1} = 40\%$  
$money precision@5 = \frac{1*400 + 0*60 + ... + 1*90}{1*400 + 1*60 + ... + 1*90} = 77.7\%$  

  
**Case 2**   
prices_resommended = [400, 60, 40, 40 , 90]  
flags = [0, 1, 0, 0 , 1]  

$precison@5 = \frac{0 + 1 + 0 +0 + 1}{1+1+1+1+1} = 40\%$  
$money precision@5 = \frac{0*400 + 1*60 + ... + 1*90}{1*400 + 1*60 + ... + 1*90} = 15.8\%$

In [ ]:
recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [521, 32, 143, 991]

In [7]:
def precision(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    return precision


def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
        
    return precision


def money_precision_at_k(recommended_list, bought_list, prices_recommended, k=5):
        
#     сделать дома
    # Лучше считать через скалярное произведение, а не цикл

    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    prices_recommended = np.array(prices_recommended)[:k]
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(recommended_list, bought_list)
    
    print(flags*prices_recommended)
    
    precision = np.sum(flags*prices_recommended) / np.sum(prices_recommended)
    
    return precision

In [ ]:
# проверяем код, сделанный в домашней работе
prices_recommended = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100] # цены рекомендованных товаров
money_precision_at_k(recommended_list, bought_list, prices_recommended, k=5)

[10  0  0 40  0]


0.3333333333333333

In [ ]:
precision(recommended_list, bought_list)

0.4

In [ ]:
precision_at_k(recommended_list, bought_list, k=3)

0.3333333333333333

### 3. Recall

*Recall* - доля рекомендованных товаров среди релевантных = Какой % купленных товаров был среди рекомендованных

- Обычно используется для моделей пре-фильтрации товаров (убрать те товары, которые точно не будем рекомендовать)

---

Recall= (# of recommended items that are relevant) / (# of relevant items)  

Recall@k = (# of recommended items @k that are relevant) / (# of relevant items)

Money Recall@k = (revenue of recommended items @k that are relevant) / (revenue of relevant items)  

    
  
**Note:** в recall@k число k обычно достаточно большое (50-200), больше чем покупок у среднестатистического юзера

In [ ]:
def recall(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall


def recall_at_k(recommended_list, bought_list, k=5):
    
    # сделать дома

    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall


def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    
    # сделать дома

    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    prices_recommended = np.array(prices_recommended)[:k]
    prices_bought = np.array(prices_bought)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(recommended_list, bought_list)
    
    print(flags*prices_recommended)
    print(np.sum(prices_bought))
    
    recall = np.sum(flags*prices_recommended) / np.sum(prices_bought)    
    
    return recall

In [ ]:
# проверяем код, сделанный в домашней работе
prices_recommended = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100] # цены рекомендованных товаров
prices_bought = [5, 15, 10, 40]
money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5)

[10  0  0 40  0]
70


0.7142857142857143

In [ ]:
recall(recommended_list, bought_list)

0.5

In [ ]:
# проверяем код, сделанный в домашней работе
recall_at_k(recommended_list,bought_list,k=10)

0.5

---

# Метрики ранжирования

Metrics: https://en.wikipedia.org/wiki/Evaluation_measures_(information_retrieval)

Если важен порядок рекомендаций.

## AP@k
AP@k - average precision at k

$$AP@k = \frac{1}{r} \sum{[recommended_{relevant_i}] * precision@k}$$

- r - кол-во релевантный среди рекомендованных
- Суммируем по всем релевантным товарам
- Зависит от порядка реокмендаций

In [10]:
recommended_list = [221,21,3234,1,234,234,234,666] #id товаров
bought_list = [1,2,3,4,5,6,7,8,9]

In [11]:
def ap_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    relevant_indexes = np.nonzero(np.isin(recommended_list, bought_list))[0]
    
    if len(relevant_indexes) == 0:
        return 0
    
    amount_relevant = len(relevant_indexes)
    
    sum_ = sum([precision_at_k(recommended_list, bought_list, k=index_relevant+1) for index_relevant in relevant_indexes])
    return sum_/amount_relevant
  
    

In [12]:
ap_k(recommended_list, bought_list, k=5)

0.25

### MAP@k

MAP@k (Mean Average Precision@k)  
Среднее AP@k по всем юзерам
- Показывает средневзвешенную точность рекомендаций

$$MAP@k = \frac{1}{|U|} \sum_u{AP_k}$$
  
|U| - кол-во юзеров

In [36]:
# теперь список из 3 пользователей
recommended_list_3_users = [[143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43], 
                    [1134, 533, 14, 4, 15, 1543, 1, 99, 27, 3345],
                    [991, 3345, 27, 533, 43, 143, 1543, 156, 1134, 11]
                           ]

bought_list_3_users = [[521, 32, 143],  # юзер 1
                       [143, 156, 991, 43, 11], # юзер 2
                       [1,2]] # юзер 3

In [37]:
def map_k(recommended_list, bought_list, k=5):
    
    # сделать дома

    amount_users = len(recommended_list_3_users)
    sum_ = sum([ap_k(recommended_list_3_users[i], bought_list_3_users[i], k=5) for i in range(amount_users)])
    result = sum_/amount_users
    
    return result

In [38]:
# проверяем код, сделанный в домашней работе
map_k(recommended_list_3_users, bought_list_3_users, k=5)

0.3333333333333333

### AUC@k
AUC для первых k наблюдений  
- Можно посчитать как sklern.metrics.roc_auc_score для топ-k предсказаний
- Показывает долю верно отранжированных товаров

### Normalized discounted cumulative gain ( NDCG@k)


$$DCG = \frac{1}{|r|} \sum_u{\frac{[bought fact]}{discount(i)}}$$  

$discount(i) = i$ if $i <= 2$,   
$discount(i) = log_2(i)$ if $i > 2$


(!) Считаем для первых k рекоммендаций   
(!) - существуют вариации с другими $discount(i)$  
i - ранк рекомендованного товара  
|r| - кол-во рекомендованных товаров 

$$NDCG = \frac{DCG}{ideal DCG}$$


$DCG@5 = \frac{1}{5}*(1 / 1 + 0 / 2 + 0 / log(3) + 1 / log(4) + 0 / log(5))$  
$ideal DCG@5 = \frac{1}{5}*(1 / 1 + 1 / 2 + 1 / log(3) + 1 / log(4) + 1 / log(5))$  

$NDCG = \frac{DCG}{ideal DCG}$

In [ ]:
# по желанию
# def ndcg_at_k():
#     pass

### Mean Reciprocal Rank ( MRR@k )


- Считаем для первых k рекоммендаций
- Найти ранк первого релевантного предсказания $k_u$
- Посчитать reciprocal rank = $\frac{1}{k_u}$

$$MRR = mean(\frac{1}{k_u})$$

In [48]:
def reciprocal_rank(recommended_list, bought_list, k=1):
    
    # сделать дома

    recommended_list = recommended_list[:k]
    
    ranks=0.
    for i, item_rec in enumerate(recommended_list):
        for item_bought in bought_list:
            if item_rec == item_bought:
                ranks += 1 / (i+1)

    result = ranks / len(recommended_list)

    return result

In [49]:
recommended_list = [2, 5, 7, 4, 11, 9, 8, 10, 12, 3]
boughted_list = [1, 3, 5, 7, 9, 11]

In [51]:
# проверяем код, сделанный в домашней работе
reciprocal_rank(recommended_list, boughted_list, k=8)

0.15